# Crawling

In [ ]:
!pip install unidecode twython tweepy beautifulsoup4 tika

In [ ]:
import warnings; warnings.simplefilter('ignore')
import tweepy, json, nltk, urllib.request, requests
from urllib.request import Request, urlopen
from twython import TwythonStreamer
from bs4 import BeautifulSoup as bs

In [ ]:
# API keys
Ck = 'Uuam2tve0yLLqEe7xuH7TaFA1' # consumer_key
Cs = 'x8Uojc74fQc3iPiNUjn8ZrQEEiwahr8lwG7TEC10RFsVZwQBoF' # consumer_secret
At = '755696906597085184-1KDWqr9ynhUi3VCP6D1xSak7uI5bw78' # access_token
As = 'cg14g3t6MlbgfZTX60cKQTPaHDwDLAhifDiSdTb495Iyv' # access_secret
'Done'

'Done'

In [ ]:
def twitter_connect(Ck, Cs, At, As, verbose = 0):
    try:
        auth = tweepy.OAuthHandler(Ck, Cs)
        auth.set_access_token(At, As)
        twitter = tweepy.API(auth, timeout=120)
        if verbose != 0:
            user = twitter.verify_credentials()
            print('Welcome "%s" you are now connected to twitter server' %user.name)
        return twitter
    except:
        print("Connection failed, please check your API keys or connection")
        return None

In [ ]:
# Koneksi ke twitter
twitter = twitter_connect(Ck, Cs, At, As, verbose = 1)

Welcome "Jayalah Negriku" you are now connected to twitter server


In [ ]:
# Max 100 tweet per "API call"
topic = 'anteraja'
N = 100 # jumlah tweet yang ingin diambil
bahasa = 'id'
T = twitter.search(q=topic, lang=bahasa, count=N, tweet_mode = 'extended')
tweets = [t._json for t in T]
print(' Berhasil mendapatkan {} tweets'.format(len(tweets)))

 Berhasil mendapatkan 100 tweets


In [ ]:
# Datanya berbentuk JSON
tweets[0]['created_at'], tweets[0]['full_text'], 

('Wed Dec 29 05:42:20 +0000 2021',
 'HAHAHAHAHAH TERNYATA PAKETNYA RUSAK🤣 DAHLAH gausah pake anteraja wkwkwkwkwk ok? ok!! https://t.co/CCIPqh8e42')

In [ ]:
tweets[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Wed Dec 29 05:42:20 +0000 2021',
 'display_text_range': [0, 84],
 'entities': {'hashtags': [],
  'media': [{'display_url': 'pic.twitter.com/CCIPqh8e42',
    'expanded_url': 'https://twitter.com/bittakuchi/status/1476066029293867008/photo/1',
    'id': 1476066019932131328,
    'id_str': '1476066019932131328',
    'indices': [85, 108],
    'media_url': 'http://pbs.twimg.com/media/FHwKOX8UUAAKs2W.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/FHwKOX8UUAAKs2W.jpg',
    'sizes': {'large': {'h': 1334, 'resize': 'fit', 'w': 750},
     'medium': {'h': 1200, 'resize': 'fit', 'w': 675},
     'small': {'h': 680, 'resize': 'fit', 'w': 382},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'type': 'photo',
    'url': 'https://t.co/CCIPqh8e42'}],
  'symbols': [],
  'urls': [],
  'user_mentions': []},
 'extended_entities': {'media': [{'display_url': 'pic.twitter.com/CCIPqh8e42',
    'expanded_url': 'https://twitter.com/

In [ ]:
# Contoh mengakses data spesifik pada tweet yang pertama:
print('tweet pertama oleh "{}" : "{}"'.format(tweets[0]['user']['screen_name'],tweets[0]['full_text']))

tweet pertama oleh "bittakuchi" : "HAHAHAHAHAH TERNYATA PAKETNYA RUSAK🤣 DAHLAH gausah pake anteraja wkwkwkwkwk ok? ok!! https://t.co/CCIPqh8e42"


In [ ]:
# Next data
last_id = T[-1]._json['id'] - 1
last_id
T = twitter.search(q=topic, lang=bahasa, count=N, tweet_mode = 'extended', max_id=last_id)
tweets.extend([t._json for t in T])

print('Jumlah data sekarang = ', len(tweets))

Jumlah data sekarang =  5100


In [ ]:
def saveTweets(tweets, file='Tweets.json'): #in Json Format
    with open(file, 'w') as f:
        for t in tweets:
            try:
                f.write(json.dumps(t)+'\n')
            except:
                pass

In [ ]:
# Menyimpan hasil crawling twitter
fileName = '/content/tweets_anteraja.json' # format tweets_[keywords].json
saveTweets(tweets,file=fileName)
print('Saved to '+fileName)

Saved to /content/tweets_anteraja.json


In [ ]:
import json
def loadTweets(file='Tweets.json'):
    f=open(file,encoding='utf-8', errors ='ignore', mode='r')
    T=f.readlines();f.close()
    for i,t in enumerate(T):
        T[i] = json.loads(t.strip())
    return T

In [ ]:
# Me-load kembali jika (misal) analisa ingin dilakukan di lain waktu
# Sengaja nama variabelnya saya bedakan (T2)
T2 = loadTweets(file='/content/tweets_anteraja.json')
print('tweet pertama oleh "{}" : "{}"'.format(T2[0]['user']['screen_name'],T2[0]['full_text']))

tweet pertama oleh "bittakuchi" : "HAHAHAHAHAH TERNYATA PAKETNYA RUSAK🤣 DAHLAH gausah pake anteraja wkwkwkwkwk ok? ok!! https://t.co/CCIPqh8e42"


In [ ]:
# Contoh mengambil hanya data tweet
D = [t['full_text'] for t in T2]
D[:15] # 5 tweet pertama

['HAHAHAHAHAH TERNYATA PAKETNYA RUSAK🤣 DAHLAH gausah pake anteraja wkwkwkwkwk ok? ok!! https://t.co/CCIPqh8e42',
 '@TokopediaCare min tolong cek DM dong, saya ada kendala, jadi penjual setuju utk return pembelian saya karna brg yg di kirim rusak utk gnti yg baru, saat saya retur pilih kurir anteraja, karna kurir tdk pickup paket akhirnya saya ke kurir sicepat taro paket dan mau gnti kurir min',
 '@anteraja_id Kemarin² aku pake anteraja huhu mantappp',
 '@miepajakenak @anteraja_id iyaaa padahal aku order di desember juga, kalo @sicepat_ekspres emang aku blm pernah ada kendala apa apa sih aman aja huft luckyyy youuu',
 '@miepajakenak Hai Kak, Ada yang bisa kami bantu Kak ? Terkait kendalanya yuk infoin nomor resinya melalui DM. Kami tunggu ya. Terima kasih ^YH',
 '@bittakuchi @anteraja_id tapi km pake tokped gratis ongkirnya masih anteraja ya? soalnya desember ini aku udah dapet sicepat terus sih',
 '@miepajakenak @anteraja_id ih kesel banget kalo misalnya dibedain kenapa ga ada warning 

# Preprocessing Func

In [ ]:
#removes html
def strip_html(text):
  soup = BeautifulSoup(text, "html.parser")
  return soup.get_text()
  
#remove between square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#eliminates contractions
def replace_contractions(text):
  """Replace contactions in string of text"""
  return contractions.fix(text)

#denoise text
def denoise_text(text):
  text = strip_html(text)
  text = remove_between_square_brackets(text)
  text = replace_contractions(text)
  return text

In [ ]:
factoryStopword = StopWordRemoverFactory()
stopword = factoryStopword.create_stop_word_remover()
stop_factory = StopWordRemoverFactory().get_stop_words()

def get_stopwords():
  docs = pd.read_csv('/content/stopwords-id.txt', header = None, names= ['stopword'])
  docs = docs['stopword'].tolist()
  return docs

stop_words = get_stopwords()

# Merge stopword
data = set(stop_factory + stop_words)

#remove stopword
def removeStopword(str):
    word_tokens = word_tokenize(str)
    filtered_sentence = [w for w in word_tokens if not w in data]
    return ' '.join(filtered_sentence)

In [ ]:
#cleaning dataset
def cleaning(str):
    #remove non-ascii
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    str = re.sub("b'|b\"",'',str)
    #remove username inside brackets
    str = re.sub('SENSITIVE-NO',' ',str) 
    #Remove additional white spaces
    str = re.sub('[\s]+', ' ', str)
    #remove URLs
    str = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    #remove punctuations
    str = re.sub(r'[^\w]|_',' ',str)
    #remove digit from string
    str = re.sub("\S*\d\S*", "", str).strip()
    #remove digit or numbers
    str = re.sub(r"\b\d+\b", " ", str)
    #to lowercase
    str = str.lower()
    return str

In [ ]:
#remove twitter symbols
def remove_symbols(str):
  #remove RT
  str = re.sub ('RT', '', str)
  #remove @username
  str = re.sub('@[^\s]+', '', str)
  return str

In [ ]:
#normalization of words
normalisasi = pd.read_csv('kamus.csv', encoding = 'latin-1')
normalisasi_map = dict(zip(normalisasi['original'], normalisasi['replacement']))
def normalizing(text):
  return ' '.join([normalisasi_map[word] if word in normalisasi_map else word for word in text.split(' ')])

In [ ]:
factoryStemmer = StemmerFactory()
stemmer = factoryStemmer.create_stemmer()

def stemming(text):
  return stemmer.stem(text)

In [ ]:
#preprocessing dataset function
def preprocessing(str):
  str = remove_symbols(str)
  str = denoise_text(str)
  str = normalizing(str)
  str = cleaning(str)
  #str = stemming(str)
  str = normalizing(str)
  str = cleaning(str)
  return str

In [ ]:
print(preprocessing('Gelatine Sheet (Bronze) / Gelatine lembaran HALAL (PER 5 LEMBAR) <br><br> @sere_nity'))
print(preprocessing('polisi tembak mati kurir narkotika psikotropika dan obat larang temu kg sabu pil inex gudang narkotika sidoarjo tribunnews.com'))
print(preprocessing('Dokter Sebut Penanganan Rekonstruksi Bayi Dina Oktavia akan Dilakukan Sampai Usia 17 tahun - Halaman 4 - www.tribunnews.com'))
print(preprocessing('Misteri Kematian Hakim Jamaluddin Saat Akan Bercerai, Istri Sudah 7 Kali Diperiksa Polisi - www.voaindonesia.com'))
print(preprocessing('uuuu neo coffee (nctzen/neotizen comment beautiful grass 🤩)'))
print(preprocessing('mengsedih'))

gelatine sheet bronze gelatine lembaran halal per lembar
polisi tembak mati kurir narkotika psikotropika dan obat larang temu kg sabu pil inex gudang narkotika sidoarjo tribunnews com
dokter sebut penanganan rekonstruksi bayi dina oktavia akan dilakukan sampai usia tahun halaman
misteri kematian hakim jamaluddin saat akan bercerai istri sudah kali diperiksa polisi
uuuu neo coffee nctzen neotizen komen cantik grass
mengsedih


# Preprocessing

In [ ]:
import csv

header = ['text']
with open('tweets_anteraja.csv', 'w') as f:
  writers = csv.writer(f, lineterminator = '\n')
  writers.writerow(header)
  for i in D:
    writers.writerow([i])

In [ ]:
import pandas as pd
data = pd.read_csv('/content/tweets_anteraja.csv') # format tweets_[keyword].csv
data.head(20)

,text
0,HAHAHAHAHAH TERNYATA PAKETNYA RUSAK🤣 DAHLAH ga...
1,"@TokopediaCare min tolong cek DM dong, saya ad..."
2,@anteraja_id Kemarin² aku pake anteraja huhu m...
3,@miepajakenak @anteraja_id iyaaa padahal aku o...
4,"@miepajakenak Hai Kak, Ada yang bisa kami bant..."
5,@bittakuchi @anteraja_id tapi km pake tokped g...
6,@miepajakenak @anteraja_id ih kesel banget kal...
7,RT @indopremier: Terdeteksi kenaikan frekuensi...
8,@bittakuchi @anteraja_id pake gratis ongkir ya...
9,"@anteraja_id Asli, pake anteraja pengiriman ja..."


In [ ]:
len(data)

5100

In [ ]:
#apply preprocessing
data['clean_text'] = data['text'].apply(preprocessing)

In [ ]:
data

,text,clean_text
0,HAHAHAHAHAH TERNYATA PAKETNYA RUSAK🤣 DAHLAH ga...,haha ternyata paketnya rusak dahlah tidak perl...
1,"@TokopediaCare min tolong cek DM dong, saya ad...",min tolong cek direct message dong saya ada ke...
2,@anteraja_id Kemarin² aku pake anteraja huhu m...,aku pakai anteraja huhu mantappp
3,@miepajakenak @anteraja_id iyaaa padahal aku o...,iya padahal aku order di desember juga kalau m...
4,"@miepajakenak Hai Kak, Ada yang bisa kami bant...",hai kak ada yang bisa kami bantu kak terkait k...
...,...,...
5095,RT @txtdrikurir: Tolong bantu usut pak oknum t...,tolong bantu usut pak oknum tersebut rekan kit...
5096,"RT @ezash: Dari kurir AnterAja kita belajar, k...",dari kurir anteraja kita belajar kalau banyakn...
5097,RT @txtdrikurir: Tolong bantu usut pak oknum t...,tolong bantu usut pak oknum tersebut rekan kit...
5098,"RT @ezash: Dari kurir AnterAja kita belajar, k...",dari kurir anteraja kita belajar kalau banyakn...


In [ ]:
# only take clean data
data = data['clean_text']

In [ ]:
# checking duplicate
data.duplicated().sum()

2073

In [ ]:
#delete duplicated data
data = data.drop_duplicates(keep = 'last')
data.duplicated().sum()

0

In [ ]:
# dropping nan
data = data.dropna()

In [ ]:
# reset index after dropping nan and deleting duplicate
data = data.reset_index(drop=True)
data

0       haha ternyata paketnya rusak dahlah tidak perl...
1       min tolong cek direct message dong saya ada ke...
2                        aku pakai anteraja huhu mantappp
3       iya padahal aku order di desember juga kalau m...
4       tapi kamu pakai tokped gratis ongkirnya masih ...
                              ...                        
3022    mencari ceperan begitu amat iya katanya idaman...
3023    pernah dengar kan digebuki polisi seasrama tap...
3024    bagaimana paket saya masa statusnya sudah dite...
3025    tolong bantu usut pak oknum tersebut rekan kit...
3026    dari kurir anteraja kita belajar kalau banyakn...
Name: clean_text, Length: 3027, dtype: object

In [ ]:
len(data)

3027

In [ ]:
# saving csv
# format tweets_[keyword]_preprocessed.csv
data.to_csv('tweets_anteraja_preprocessed.csv', sep = ',', encoding = 'utf-8', index = False) 